# RDFLIB

In [1]:
import re 

def cleaning_label(label):
    pattern = r"[&\-\(\)\\/]"  
    # Replace them with a space  
    label = re.sub(pattern, ' ', label)

    return label

In [45]:
import rdflib
from rdflib import Graph, URIRef, Namespace
from rdflib.namespace import SKOS, RDF, DCTERMS, OWL, XSD
from rdflib import Literal as LiteralRDF

def add_concept(taxonomy: Graph, namespace: str, concept:dict, level: int) -> None:
    uri = get_uri(namespace, concept, level)

    # Concept
    taxonomy.add((URIRef(uri), RDF.type, SKOS.Concept)) 

    #taxonomy.add((URIRef(uri), SKOS.altLabel, LiteralRDF("altLabel", lang="fr")))
    taxonomy.add((URIRef(uri), SKOS.broader, URIRef(get_uri(namespace, concept, level-1))))
    taxonomy.add((URIRef(uri), SKOS.definition, LiteralRDF(concept[f"Description Catégorie L{level}"], lang="fr")))
    taxonomy.add((URIRef(uri), DCTERMS.identifier, LiteralRDF(concept[f"ID catégorie L{level}"])))
    taxonomy.add((URIRef(uri), SKOS.inScheme, URIRef(get_uri(namespace, concept, 2))))
    #taxonomy.add((URIRef(uri), DCTERMS.isReplacedBy, URIRef(get_uri(namespace, concept, level-1))))
    taxonomy.add((URIRef(uri), SKOS.prefLabel, LiteralRDF(cleaning_label(concept[f"Titre Catégorie L{level}"]), lang="fr")))
    #taxonomy.add((URIRef(uri), DCTERMS.replaces, URIRef(get_uri(namespace, concept, level-1))))
    taxonomy.add((URIRef(uri), URIRef("http://publications.europa.eu/ontology/euvoc#status"), URIRef("http://publications.europa.eu/resource/authority/concept-status/CURRENT")))
    taxonomy.add((URIRef(uri), OWL.versionInfo, LiteralRDF("0.0.1")))

def add_topConcept(taxonomy: Graph, namespace: str, concept:dict, level: int) -> None:
    uri = get_uri(namespace, concept, level)
    # Concept
    taxonomy.add((URIRef(uri), RDF.type, SKOS.Concept)) 

    #taxonomy.add((URIRef(uri), SKOS.altLabel, LiteralRDF("", lang="fr")))
    taxonomy.add((URIRef(uri), SKOS.definition, LiteralRDF(concept[f"Description Catégorie L{level}"], lang="fr")))
    #taxonomy.add((URIRef(uri), DCTERMS.identifier, LiteralRDF(concept[f"ID catégorie L{level}"])))
    taxonomy.add((URIRef(uri), SKOS.inScheme, URIRef(get_uri(namespace, concept, 2))))
    #taxonomy.add((URIRef(uri), DCTERMS.isReplacedBy, URIRef(get_uri(namespace, concept, level-1))))
    taxonomy.add((URIRef(uri), SKOS.prefLabel, LiteralRDF(cleaning_label(concept[f"Titre Catégorie L{level}"]), lang="fr")))
    #taxonomy.add((URIRef(uri), DCTERMS.replaces, URIRef(get_uri(namespace, concept, level-1))))
    taxonomy.add((URIRef(uri), URIRef("http://publications.europa.eu/ontology/euvoc#status"), URIRef("http://publications.europa.eu/resource/authority/concept-status/CURRENT")))
    taxonomy.add((URIRef(uri), SKOS.topConceptOf, URIRef(get_uri(namespace, concept, 2))))
    taxonomy.add((URIRef(uri), OWL.versionInfo, LiteralRDF("0.0.1")))

    taxonomy.add((URIRef(get_uri(namespace, concept, 2)), SKOS.hasTopConcept, URIRef(uri)))

def add_conceptScheme(taxonomy: Graph, namespace: str, concept:dict, level: int) -> None:
    uri = get_uri(namespace, concept, level)

    # Concept
    taxonomy.add((URIRef(uri), RDF.type, SKOS.ConceptScheme))
    
    taxonomy.add((URIRef(uri), DCTERMS.created, LiteralRDF("2024-12-18", datatype=XSD.date)))
    #taxonomy.add((URIRef(uri), DCTERMS.issued, LiteralRDF("")))
    #taxonomy.add((URIRef(uri), DCTERMS.modified, LiteralRDF("")))
    taxonomy.add((URIRef(uri), DCTERMS.identifier, LiteralRDF(concept[f"ID catégorie L{level}"])))
    #taxonomy.add((URIRef(uri), DCTERMS.isReplacedBy, URIRef(get_uri(namespace, concept, level-1))))
    taxonomy.add((URIRef(uri), SKOS.prefLabel, LiteralRDF(cleaning_label(concept[f"Titre Catégorie L{level}"]), lang="fr")))
    #taxonomy.add((URIRef(uri), DCTERMS.replaces, URIRef(get_uri(namespace, concept, level-1))))
    taxonomy.add((URIRef(uri), DCTERMS.title, LiteralRDF(concept[f"Titre Catégorie L{level}"], lang="fr")))
    taxonomy.add((URIRef(uri), OWL.versionInfo, LiteralRDF("0.0.1")))

def get_uri(namespace: str, concept:dict, level: int) -> str:
    slug = concept[f"Slug Catégorie L{level}"].lower().replace(" ", "_")
    uri = namespace + slug

    return uri

In [3]:
import pandas as pd 
import json

def rename_columns(excel: pd.DataFrame, excel_info: json, level: int) -> None:

    if excel_info["Information by level"]["altLabel"] == "":
        excel.rename(columns={excel_info["Information by level"]["Concept"] + str(level): f"Slug Catégorie L{level}", 
                            excel_info["Information by level"]["prefLabel"] + str(level): f"Titre Catégorie L{level}", 
                            excel_info["Information by level"]["Definition"] + str(level): f"Definition Catégorie L{level}"}, inplace=True)
    else:
        excel.rename(columns={excel_info["Information by level"]["Concept"] + str(level): f"Slug Catégorie L{level}", 
                            excel_info["Information by level"]["prefLabel"] + str(level): f"Titre Catégorie L{level}", 
                            excel_info["Information by level"]["Definition"] + str(level): f"Definition Catégorie L{level}",
                            excel_info["Information by level"]["altLabel"] + str(level): f"Autre Titre Catégorie L{level}"}, inplace=True)


In [46]:
import pandas as pd
import json

EXCEL_PATH = r"C:\Users\ecaudron001\Downloads\2024-10-08_D4W_taxxo-complete.xlsx"
NAMESPACE = "http://www.data4wallonia.be/Test-Taxonomy#"

# Open and read the JSON file
with open('excel_info.json', 'r', encoding="utf-8") as file:
    EXCEL_INFO = json.load(file)

# Read taxonomy from excel
taxo_excel = pd.read_excel(EXCEL_PATH)

# Create rdf version of taxonomy
taxo_graph = Graph()
taxo_graph.bind("d4w", NAMESPACE)
taxo_graph.bind("status", "http://publications.europa.eu/resource/authority/concept-status/")
taxo_graph.bind("eurovoc", "http://publications.europa.eu/ontology/euvoc#")

# Add categories to the rdf
for level in range(int(EXCEL_INFO["highest level"]), int(EXCEL_INFO["lowest level"])):

    try:     
        unique_concepts = taxo_excel.drop_duplicates(subset=f"Titre Catégorie L{level}")
        
        for index in unique_concepts.index:
            if level > 3: 
                add_concept(taxo_graph, NAMESPACE, unique_concepts.loc[index], level)
            elif level == 3: 
                add_topConcept(taxo_graph, NAMESPACE, unique_concepts.loc[index], level)
            else: 
                add_conceptScheme(taxo_graph, NAMESPACE, unique_concepts.loc[index], level)
    except:
        rename_columns(taxo_excel, EXCEL_INFO, level)
        unique_concepts = taxo_excel.drop_duplicates(subset=f"Titre Catégorie L{level}")
        
        for index in unique_concepts.index:
            if level > 3: 
                add_concept(taxo_graph, NAMESPACE, unique_concepts.loc[index], level)
            elif level == 3: 
                add_topConcept(taxo_graph, NAMESPACE, unique_concepts.loc[index], level)
            else: 
                add_conceptScheme(taxo_graph, NAMESPACE, unique_concepts.loc[index], level)

# Save rdf file
taxo_graph.serialize("taxo_2_rdf.ttl", format="ttl")
    

<Graph identifier=N16bdf2ba96134085b4b3f5b8d32d7bc4 (<class 'rdflib.graph.Graph'>)>

In [50]:
import logging
import requests  

turtle_data = taxo_graph.serialize(format="turtle")#.decode("utf-8")  
  
# Prepare the API request payload  
payload = {  
    "contentToValidate": turtle_data,  
    "contentSyntax": "text/turtle",  
    "validationType": "v1.0.0"  
}  

# Send the POST request  
response = requests.post("http://localhost:8080/shacl/d4wta-ap/api/validate", json=payload)  

# Check the response  
if response.status_code == 200:  
    logging.info("Validation successful")
    
    if response.json().get("sh:conforms"):
        logging.info("No errors in the taxonomy")
        print("No error in the taxonomy")
    else: 
        logging.info("Errors detected:\n" + response.text) 
        print("Errors detected:\n" + response.text)
else:  
    print("Validation failed:", response.status_code, response.text)

Errors detected:
{
    "@graph": [
        {
            "@id": "_:b0",
            "sh:resultMessage": "Property needs to have at least 1 value",
            "sh:resultPath": {
                "@id": "dc:identifier"
            },
            "sh:focusNode": {
                "@id": "d4w:filiere-du-bois"
            },
            "sh:sourceShape": {
                "@id": "https://DigitalWallonia.github.io/D4WTA-AP/releases/1.0.0/#ConceptShape/6c7c1bd2ab3b7761e5be759a078a9fd93f411a57"
            },
            "sh:sourceConstraintComponent": {
                "@id": "sh:MinCountConstraintComponent"
            },
            "sh:resultSeverity": {
                "@id": "sh:Violation"
            },
            "@type": "sh:ValidationResult"
        },
        {
            "@id": "_:b1",
            "sh:resultMessage": "Property needs to have at least 1 value",
            "sh:resultPath": {
                "@id": "dc:identifier"
            },
            "sh:focusNode": {
       

In [51]:
from tqdm import tqdm

for index in tqdm([1], desc="SHACL validation"):
    payload = {  
    "contentToValidate": turtle_data,  
    "contentSyntax": "text/turtle",  
    "validationType": "v1.0.0"  
}  

    # Send the POST request  
    response = requests.post("http://localhost:8080/shacl/d4wta-ap/api/validate", json=payload)  

# Check the response  
if response.status_code == 200:  
    logging.info("Validation successful")
    
    if response.json().get("sh:conforms"):
        logging.info("No errors in the taxonomy")
        print("No error in the taxonomy")
    else: 
        logging.info("Errors detected:\n" + response.text) 
        print("Errors detected:\n" + response.text)
else:  
    print("Validation failed:", response.status_code, response.text)

SHACL validation: 100%|██████████| 1/1 [00:00<00:00,  2.76it/s]

Errors detected:
{
    "@graph": [
        {
            "@id": "_:b0",
            "sh:resultMessage": "Property needs to have at least 1 value",
            "sh:resultPath": {
                "@id": "dc:identifier"
            },
            "sh:focusNode": {
                "@id": "d4w:nature"
            },
            "sh:sourceShape": {
                "@id": "https://DigitalWallonia.github.io/D4WTA-AP/releases/1.0.0/#ConceptShape/6c7c1bd2ab3b7761e5be759a078a9fd93f411a57"
            },
            "sh:sourceConstraintComponent": {
                "@id": "sh:MinCountConstraintComponent"
            },
            "sh:resultSeverity": {
                "@id": "sh:Violation"
            },
            "@type": "sh:ValidationResult"
        },
        {
            "@id": "_:b1",
            "sh:resultMessage": "Property needs to have at least 1 value",
            "sh:resultPath": {
                "@id": "dc:identifier"
            },
            "sh:focusNode": {
                

# OWLREADY2

In [33]:
from owlready2 import *

onto = get_ontology("http://test.org/onto2.owl")

### Creating the model

In [34]:
NAMESPACE = "http://www.data4wallonia.be/Test-Taxonomy#"

with onto:
    # Define a new namespace for SKOS
    skos = onto.get_namespace("http://www.w3.org/2004/02/skos/core#")
    owl = onto.get_namespace("http://www.w3.org/2002/07/owl#")
    dcterms = onto.get_namespace("http://purl.org/dc/terms/")
    eurovoc = onto.get_namespace("http://publications.europa.eu/ontology/euvoc#")
    status = onto.get_namespace("http://publications.europa.eu/resource/authority/concept-status/")
    d4w = onto.get_namespace(NAMESPACE)

    # Create the skos:ConceptScheme class
    class ConceptScheme(Thing):
        namespace = skos
    # Create the skos:Concept class
    class Concept(Thing):
        namespace = skos

    # Create the skos:ConceptScheme attributes
    class created(DataProperty):
        namespace = dcterms
        domain = [ConceptScheme]
        range = [datetime.date]
    class identifier(DataProperty):
        namespace = dcterms
        domain = [ConceptScheme]
        range = [str]
    class issued(DataProperty):
        namespace = dcterms
        domain = [ConceptScheme]
        range = [datetime.date]
    class modified(DataProperty):
        namespace = dcterms
        domain = [ConceptScheme]
        range = [datetime.date]
    class prefLabel(DataProperty):
        namespace = skos
        domain = [ConceptScheme, Concept]
        range = [str]
    class title(DataProperty):
        namespace = dcterms
        domain = [ConceptScheme]
        range = [str]
    class versionInfo(DataProperty):
        namespace = owl
        domain = [ConceptScheme, Concept]
        range = [str]

    # Create the skos:ConceptScheme properties
    class ReplacedBy(ObjectProperty):
        skos = dcterms
        domain = [ConceptScheme, Concept]
        range = [ConceptScheme, Concept]
    class Replaces(ObjectProperty):
        namespace = dcterms
        domain = [ConceptScheme, Concept]
        range = [ConceptScheme, Concept]
    class hasTopConcept(ObjectProperty):
        namespace = skos
        domain = [ConceptScheme]
        range = [Concept]

    # Create the skos:Concept attributes
    class altLabel(DataProperty):
        namespace = skos
        domain = [Concept]
        range = [str]
    class definition(DataProperty):
        namespace = skos
        domain = [Concept]
        range = [str]


    # Define the skos:inScheme property
    class inScheme(ObjectProperty):
        namespace = skos
        domain = [Concept]  # Specifies that the domain is skos:Concept
        range = [ConceptScheme]  # Specifies that the range is skos:ConceptScheme
    class topConceptOf(ObjectProperty):
        namespace = skos
        domain = [Concept]
        range = [ConceptScheme]
    class broader(ObjectProperty):
        namespace = skos
        domain = [Concept]
        range = [Concept]
    class status(ObjectProperty):
        namespace = eurovoc
        domain = [Concept]
        range = [Concept]

### Instantiating

In [35]:
def add_concept(onto: OWL, namespace: str, concept:dict, level: int) -> None:
    with onto:
        Concept = skos.Concept(concept[f"Slug Catégorie L{level}"])
        
        # Concept.altLabel.append(LiteralRDF("altLabel", lang="fr"))
        Concept.broader = [skos.Concept(concept[f"Slug Catégorie L{level-1}"])]
        Concept.definition.append(locstr(concept[f"Description Catégorie L{level}"], "fr"))
        Concept.inScheme.append(skos.ConceptScheme(concept[f"Slug Catégorie L2"]))
        # Concept.isReplacedBy.append(skos.concept())
        Concept.prefLabel.append(locstr(concept[f"Titre Catégorie L{level}"], "fr"))
        # Concept.replaces(skos.concept())
        Concept.status.append(skos.Concept(r"http://publications.europa.eu/resource/authority/concept-status/CURRENT"))
        Concept.versionInfo.append("0.0.1")

def add_topConcept(onto: OWL, namespace: str, concept:dict, level: int) -> None:
    with onto:
        Concept = skos.Concept(concept[f"Slug Catégorie L{level}"])
        
        # Concept.altLabel.append(LiteralRDF("altLabel", lang="fr"))
        Concept.definition.append(locstr(concept[f"Description Catégorie L{level}"], "fr"))
        Concept.inScheme.append(skos.ConceptScheme(concept[f"Slug Catégorie L2"]))
        # Concept.isReplacedBy.append(skos.concept())
        Concept.prefLabel.append(locstr(concept[f"Titre Catégorie L{level}"], "fr"))
        # Concept.replaces(skos.concept())
        Concept.status.append(skos.Concept(r"http://publications.europa.eu/resource/authority/concept-status/CURRENT"))
        Concept.topConceptOf.append(skos.ConceptScheme(concept[f"Slug Catégorie L2"]))
        Concept.versionInfo.append("0.0.1")

def add_conceptScheme(onto: OWL, namespace: str, concept:dict, level: int) -> None:
    with onto:
        conceptScheme = skos.ConceptScheme(concept[f"Slug Catégorie L{level}"])

        conceptScheme.created.append(datetime.date(2024, 12, 18))
        # conceptScheme.issued.append(LiteralRDF("2024-12-18", datatype=XSD.date))
        # conceptScheme.modified.append(LiteralRDF("2024-12-18", datatype=XSD.date))
        conceptScheme.identifier.append(locstr(concept[f"ID catégorie L{level}"], "fr"))
        # conceptScheme.isReplacedBy.append(skos.ConceptScheme())
        conceptScheme.prefLabel.append(locstr(concept[f"Titre Catégorie L{level}"],"fr")) 
        # conceptScheme.replaces(skos.ConceptScheme())
        conceptScheme.title.append(locstr(concept[f"Titre Catégorie L{level}"],"fr"))
        conceptScheme.versionInfo.append("0.0.1")

In [36]:
import pandas as pd
import json

EXCEL_PATH = r"C:\Users\ecaudron001\Downloads\2024-10-08_D4W_taxxo-complete.xlsx"
NAMESPACE = "http://www.data4wallonia.be/Test-Taxonomy#"

# Open and read the JSON file
with open('excel_info.json', 'r', encoding="utf-8") as file:
    EXCEL_INFO = json.load(file)

# Read taxonomy from excel
taxo_excel = pd.read_excel(EXCEL_PATH)

# Add categories to the rdf
for level in range(int(EXCEL_INFO["highest level"]), int(EXCEL_INFO["lowest level"])):

    unique_concepts = taxo_excel.drop_duplicates(subset=f"Titre Catégorie L{level}")
    
    for index in unique_concepts.index:
        if level > 3: 
            add_concept(onto, NAMESPACE, unique_concepts.loc[index], level)
        elif level == 3: 
            add_topConcept(onto, NAMESPACE, unique_concepts.loc[index], level)
        else: 
            add_conceptScheme(onto, NAMESPACE, unique_concepts.loc[index], level)

In [37]:
#onto.bind("owl", "http://www.w3.org/2002/07/owl#", override=True)
onto.save(file = "d4w_taxo2", format = "rdfxml")

In [32]:
onto.destroy()

In [2]:
import yaml  

def load_config(config_path):
    with open(config_path, 'r') as file:  
        config = yaml.safe_load(file)  
    return config 

config = load_config("../config.yaml") 


In [4]:
config['transformation']['rules']['changes'][0]["changelabel"]["from"]

'&-()/'

In [9]:
import re  
  
def build_exclusion_pattern(chars_to_exclude):  
    # Escape each character for regex use  
    escaped_chars = [re.escape(char) for char in chars_to_exclude]  
      
    # Join them into a string with no separator  
    char_class = ''.join(escaped_chars)  
      
    # Build the regex pattern with negation  
    pattern = f'[{char_class}]'  
      
    return pattern  
  
# List of characters to exclude  
exclude_chars = config['transformation']['rules']['changes'][0]["changelabel"]["from"]  
  
# Build the regex pattern  
pattern = build_exclusion_pattern(exclude_chars)  
  
# Compile the pattern  
regex = re.compile(pattern)  
  
# Example usage: Find all characters not in the exclusion list  
test_string = "abc123.xyz"  
matches = regex.findall(test_string)  
  
print(f"Excluded characters: {exclude_chars}")  
print(f"Regex pattern: {pattern}")  
print(f"Original string: {test_string}")  
print(f"Matched characters: {matches}")  

Excluded characters: &-()/
Regex pattern: [\&\-\(\)/]
Original string: abc123.xyz
Matched characters: []


In [10]:
if regex.findall(test_string):
    print("here")

In [11]:
from rdflib import Graph
taxo_graph = Graph()
taxo_graph.parse("output/taxo_2_rdf.ttl", format="ttl")

<Graph identifier=N0301f28b17824c09b4b5c14562b5bab4 (<class 'rdflib.graph.Graph'>)>

In [23]:
from rdflib.namespace import RDF, SKOS
pref_labels = []
for concept in taxo_graph.subjects(RDF.type, SKOS.Concept):  
    for label, definition in zip(taxo_graph.objects(concept, SKOS.prefLabel),taxo_graph.objects(concept, SKOS.definition)):  
        print(f"  {concept}: {label} - {definition}")
        pref_labels.append(str(label))


  http://www.data4wallonia.be/Test-Taxonomy#acces-au-wifi: Accéder à du WiFi - Avoir accès à une connection Internet.
  http://www.data4wallonia.be/Test-Taxonomy#access-management: Access Management - /
  http://www.data4wallonia.be/Test-Taxonomy#accompagnement-developpement: Accompagnement   développement - Initiatives et acteurs impliqués dans la création, l'incubation, l'accélération et l'accompagnement (coaching, rédaction de business plans, etc.).
  http://www.data4wallonia.be/Test-Taxonomy#accompagnement-made-different: Accompagnement Made Different Digital Wallonia - Personnes de contact faisant partie de l'équipe d'accompagnement pour le programme Made Different Digital Wallonia.
  http://www.data4wallonia.be/Test-Taxonomy#acteur-de-la-promotion-des-metiers: Acteur de la promotion des métiers - Acteur de la promotion ou de sensibilisation des métiers : tout acteur visé par des actions de réseautage ou partenariat liées à la promotion des métiers ou sensibilisation aux STEAM.
  

In [17]:
list(taxo_graph.objects(concept, SKOS.prefLabel))

[rdflib.term.Literal('Accéder à du WiFi', lang='fr')]

In [35]:
from sentence_transformers import SentenceTransformer  
from sklearn.metrics.pairwise import cosine_similarity  
import pandas as pd  
import numpy as np 

# Load a pre-trained Sentence-BERT model  
model_name = "paraphrase-MiniLM-L6-v2"  # You can choose a different SBERT model if needed  
model = SentenceTransformer(model_name)  

# Generate embeddings for all prefLabels  
embeddings = model.encode(pref_labels, convert_to_tensor=True)  

# Compute the cosine similarity matrix  
similarity_matrix = cosine_similarity(embeddings.cpu().numpy())  

# Convert the similarity matrix to a DataFrame for easier handling  
df_similarity = pd.DataFrame(similarity_matrix, index=pref_labels, columns=pref_labels)  

# Display the similarity matrix  
print(df_similarity)  

# Find all pairs with similarity above the lower threshold and below the upper threshold  
def get_similar_pairs_in_range(df, lower_threshold=0.75, upper_threshold=1.0):  
    # Get the upper triangle of the similarity matrix, excluding the diagonal  
    df_triu = df.where(~np.triu(np.ones(df.shape), k=1).astype(bool))  

    # Stack the matrix and filter pairs within the specified range  
    pairs_in_range = df_triu.stack().loc[lambda x: (x > lower_threshold) & (x < upper_threshold)]  

    return pairs_in_range  

# Set your similarity range  
lower_threshold = 0.75  
upper_threshold = 1.0  
similar_pairs = get_similar_pairs_in_range(df_similarity, lower_threshold, upper_threshold)  

print(f"Pairs with similarity above {lower_threshold} and below {upper_threshold}:")  
for (label1, label2), similarity in similar_pairs.items():  
    print(f"{label1} - {label2}: {similarity}") 

c:\Users\ecaudron001\Documents\GitHub\spreadsheet-to-rdf\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No sentence-transformers model found with name sentence-transformers/paraphrase-MiniLM-L6-v2. Creating a new one with mean pooling.


SSLError: (MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /sentence-transformers/paraphrase-MiniLM-L6-v2/resolve/main/adapter_config.json (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1007)')))"), '(Request ID: 8e72a008-1d26-4a68-94b5-7d02ffd51f6b)')

In [32]:
most_similar_pairs.tail()

Foire de Libramont 2018                Foire de Libramont 2019                    0.805667
Marché industrie manufacturière        Secteur industrie manufacturière           0.804889
Marché industrie pharmaceutique        Secteur industrie pharmaceutique           0.804889
E santé                                Marché santé                               0.804360
Chanvre   Chaux chanvre en vrac à sec  Chanvre   Chaux chanvre en vrac projeté    0.803855
dtype: float64